<a href="https://colab.research.google.com/github/AGlade7/digitClassifier/blob/main/digitClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [2]:
train = datasets.MNIST(root = "data", download = True, train = True, transform = ToTensor())
dataset = DataLoader(train, 32)

100%|██████████| 9912422/9912422 [00:00<00:00, 111823686.70it/s]


Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 15068502.78it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 61397093.42it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 20865858.45it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



In [16]:
for batch in dataset:
    X, y = batch
    print(type(X))
    break

<class 'torch.Tensor'>


In [22]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 32, (3,3)),
            nn.ReLU(),
            nn.Conv2d(32, 64, (3,3)),
            nn.ReLU(),
            nn.Conv2d(64, 64, (3,3)),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(64*(22)*(22), 10)
        )

    def forward(self, x):
        return self.model(x)

In [37]:
clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-3)

In [38]:
loss_fn = nn.CrossEntropyLoss()

In [39]:
from PIL import Image
import numpy as np

if __name__ == "__main__":
    for epoch in range(10): # train for 10 epochs
        for batch in dataset:
            X,y = batch
            X, y = X.to('cuda'), y.to('cuda')
            yhat = clf(X)
            loss = loss_fn(yhat, y)

            # Apply backprop
            opt.zero_grad()
            loss.backward()
            opt.step()

        print(f"Epoch: {epoch + 1} loss is {loss.item()}")

    with open('model_state.pt', 'wb') as f:
        torch.save(clf.state_dict(), f)

    with open('model_state.pt', 'rb') as f:
         clf.load_state_dict(torch.load(f))

    img = Image.open('2.jpg')
    img = np.array(img)
    img.resize(28,28)
    img_tensor = ToTensor()(img).unsqueeze(0).to('cuda')

    print(torch.argmax(clf(img_tensor)))

Epoch: 1 loss is 0.012851343490183353
Epoch: 2 loss is 0.010246719233691692
Epoch: 3 loss is 0.0015443033771589398
Epoch: 4 loss is 4.0345712477574125e-05
Epoch: 5 loss is 0.00013715615205001086
Epoch: 6 loss is 1.1561466635612305e-05
Epoch: 7 loss is 0.00015856995014473796
Epoch: 8 loss is 0.0002038415550487116
Epoch: 9 loss is 3.086058495682664e-05
Epoch: 10 loss is 1.9510098354658112e-05
tensor(2, device='cuda:0')
